# Data Modeling:
lets make model for predicting Number of pickups at given 10 mint time interval within perticular cluster.

###  ML Problem Formulation:-
Time-series forecasting and Regression:-


### Feature Selecation:-
As for baseline models and some ML model, we will take previous interval pickups( P(t-1), P(t-2).....P(t-n)) as feature to predict pickups at P(t)
where n is hyperparameter, need to find best n.
<p>In later part we can use other parameter like weeks, weekends, ratio etc feature to improve our model, as of now will try only preveous pickups details</p>

###  Performance metrics
1. Mean Absolute percentage error.
https://en.wikipedia.org/wiki/Mean_absolute_percentage_error

In [2]:
# Import all used packages
import pandas as pd
import folium #open street map

import datetime
import time
import numpy as np
import matplotlib
matplotlib.use('nbagg') # for more intractive visualization like zoom out
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import gpxpy.geo #Get the haversine distance
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Load data
file = "../output/clean_train_test_yellow_tripdata_2015-01.csv"
jan_data = pd.read_csv(file)

In [4]:
print("Jan2019 data row and columns informantion")
print(jan_data.shape)

Jan2019 data row and columns informantion
(12371076, 4)


In [5]:
jan_data.head()

,pickup_time_unix,trip_distance,pickup_cluster,10_min_bin
0,1.420051e+09,4.00,8,33
1,1.420051e+09,0.80,7,32
2,1.420051e+09,1.58,29,32
3,1.420051e+09,2.57,38,32
4,1.420051e+09,2.50,28,32


### Train Test Split:-
<p>1-jan-2015 to 19-jan-2015 --> Train</p>
20-jan-2016 to 31-jan-2015 --->Test

In [6]:
unix_time_20_jan = 1421712000
jan_data_train = jan_data[jan_data.pickup_time_unix < 1421712000].copy()
jan_data_test = jan_data[jan_data.pickup_time_unix > 1421712000].copy()
print("Training data size =  "+str(len(jan_data_train)))
print("Testing data size =  "+str(len(jan_data_test)))

Training data size =  7715505
Testing data size =  4655569


In [7]:
#Refer:https://www.unixtimestamp.com/
start_unix_time = 1420070400
#1575158400 : 2019-01-12 00:00:00

jan_data_train["10_min_bin"] = jan_data_train["pickup_time_unix"].apply(lambda x : int(abs((x-start_unix_time)/(60*10))))
jan_2015_groupby_10m_train = jan_data_train[["pickup_cluster","10_min_bin","trip_distance"]].groupby(["pickup_cluster","10_min_bin"]).count()
jan_2015_groupby_10m_train.reset_index(inplace=True)
jan_2015_groupby_10m_train.rename(columns = {"trip_distance":"trip_count"},inplace=True)
jan_2015_groupby_10m_train["trip_count"] = jan_2015_groupby_10m_train.trip_count.astype(float)

jan_data_test["10_min_bin"] = jan_data_test["pickup_time_unix"].apply(lambda x : int(abs((x-start_unix_time)/(60*10))))
jan_2015_groupby_10m_test = jan_data_test[["pickup_cluster","10_min_bin","trip_distance"]].groupby(["pickup_cluster","10_min_bin"]).count()
jan_2015_groupby_10m_test.reset_index(inplace=True)
jan_2015_groupby_10m_test.rename(columns = {"trip_distance":"trip_count"},inplace=True)
jan_2015_groupby_10m_test["trip_count"]= jan_2015_groupby_10m_test.trip_count.astype(float)
jan_2015_groupby_10m_test.head()

,pickup_cluster,10_min_bin,trip_count
0,0,2736,19.0
1,0,2737,13.0
2,0,2738,19.0
3,0,2739,22.0
4,0,2740,27.0


###  Zero Filling:-


In [19]:
print("Befor Zero filling Training set size : "+str(len(jan_2015_groupby_10m_train)))
jan_2015_groupby_10m_train
new_data_dict =[]
data_index = list(jan_2015_groupby_10m_train.groupby(["pickup_cluster","10_min_bin"]).count().index.values)
for clust in range(30):
    for slot in range(2735):
        if not (clust,slot) in data_index:
            new_data_dict.append([clust,slot,0])

new_data = pd.DataFrame(new_data_dict, columns= jan_2015_groupby_10m_train.columns)
jan_2015_groupby_10m_train = jan_2015_groupby_10m_train.append(new_data,ignore_index=True)
print("After Zero filling Training set size : "+str(len(jan_2015_groupby_10m_train)))

####  Feature extraction:-
Get Feature of P(t) 5 previous interval pickups  as P(t-1), P(t-2),P(t-3), P(t-4),P(t-5)

In [30]:
def get_features(clusters):
    val_feature = []
    val_label = []
    for grp, val_df in clusters:
        val_df.sort_values("10_min_bin",inplace = True)
        val_feature.extend(np.array([ list(val_df.trip_count[i:i+5]) for i in range(len(val_df)-6)]))
        val_label.extend( [val_df.trip_count.iloc[i+6] for i in range(len(val_df)-6)])
    return val_feature, val_label
    

In [31]:
x_train, y_train = get_features(jan_2015_groupby_10m_train.groupby("pickup_cluster"))
x_test, y_test = get_features(jan_2015_groupby_10m_test.groupby("pickup_cluster"))


### mean absolute percentage error

In [39]:
#mean absolute percentage error
#def mean_absolute_percentage_error(y_true, y_pred): 
#    y_true, y_pred = np.array(y_true), np.array(y_pred)
#    return np.mean(np.abs((y_true - y_pred) / (y_true+0.00001)))*100

In [46]:
#mean absolute percentage error
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = np.mean(y_true)
    return np.mean(np.abs((y_true - y_pred) /y_true))*100

## Modeling

## 1. Linear Regression

In [47]:
lr_reg=LinearRegression().fit(x_train, y_train)

y_pred = lr_reg.predict(x_test)
lr_test_predictions = [round(value) for value in y_pred]
y_pred = lr_reg.predict(x_train)
lr_train_predictions = [round(value) for value in y_pred]

lr_mape_train = mean_absolute_percentage_error(y_train,lr_train_predictions)
lr_mape_test = mean_absolute_percentage_error(y_test,lr_test_predictions)
print("Linear Regression --> Train MAPE : "+str(lr_mape_train))
print("Linear Regression --> Test MAPE : "+str(lr_mape_test))

Linear Regression --> Train MARE : 78.44930023588755
Linear Regression --> Test MARE : 78.38374686942757


## 2. Random Forest Regressor

In [49]:
rf = RandomForestRegressor(max_features='sqrt',min_samples_leaf=4,min_samples_split=3,n_estimators=500, n_jobs=-1)
rf.fit(x_train, y_train)

y_pred = rf.predict(x_test)
rf_test_predictions = [round(value) for value in y_pred]
y_pred = rf.predict(x_train)
rf_train_predictions = [round(value) for value in y_pred]

rf_mape_train = mean_absolute_percentage_error(y_train,rf_train_predictions)
rf_mape_test = mean_absolute_percentage_error(y_test,rf_test_predictions)
print("Random Forest Regressor --> Train MAPE : "+str(rf_mape_train))
print("Random Forest Regressor --> Test MAPE : "+str(rf_mape_test))

Random Forest Regressor --> Train MARE : 79.16781613968288
Random Forest Regressor --> Test MARE : 79.03402484176057


## 3. XGBoost Regressor

In [51]:
x_model = xgb.XGBRegressor(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=3,
 gamma=0,
 subsample=0.8,
 reg_alpha=200, reg_lambda=200,
 colsample_bytree=0.8,nthread=4)
x_model.fit(x_train, y_train)

y_pred = x_model.predict(x_test)
xgb_test_predictions = [round(value) for value in y_pred]
y_pred = x_model.predict(x_train)
xgb_train_predictions = [round(value) for value in y_pred]

xgb_mape_train = mean_absolute_percentage_error(y_train,xgb_train_predictions)
xgb_mape_test = mean_absolute_percentage_error(y_test,xgb_test_predictions)
print("XGB Regressor --> Train MAPE : "+str(xgb_mape_train))
print("XGB Regressor --> Test MAPE : "+str(xgb_mape_test))

XGB Regressor --> Train MARE : 79.04502153396606
XGB Regressor --> Test MARE : 79.03506755828857


## 4. Light GBM Regressor

In [57]:
lgbm = LGBMRegressor(
 learning_rate =0.01,
 n_estimators=1000,
 max_depth=2,
 min_child_weight=3,
 gamma=0,
 subsample=0.8,
 reg_alpha=200, reg_lambda=200,
 colsample_bytree=0.8,nthread=4)
lgbm.fit(x_train, y_train)

y_pred = lgbm.predict(x_test)
lgbm_test_predictions = [round(value) for value in y_pred]
y_pred = lgbm.predict(x_train)
lgbm_train_predictions = [round(value) for value in y_pred]

lgbm_mape_train = mean_absolute_percentage_error(y_train,lgbm_train_predictions)
lgbm_mape_test = mean_absolute_percentage_error(y_test,lgbm_test_predictions)
print("LGBM Regressor --> Train MAPE : "+str(lgbm_mape_train))
print("LGBM Regressor --> Test MAPE : "+str(lgbm_mape_test))

Linear Regression --> Train MARE : 78.65115363295799
Linear Regression --> Test MARE : 78.58739364218933


## Save model:-


In [60]:
import pickle

In [66]:
lgbm_model_file = "../model/lgbm_model.sav"
with open(lgbm_model_file,'wb') as file_obj:
    pickle.dump(lgbm, file_obj)

## Result Observation:-

<p>1.We can see Best MAPE we got from Light GBM model.</p>
<p>2. But still its looks high, we can go for hyperparameter tuning to improve model performance.</p>
<p>3. And Here we are using just one month data with feature as previous 5 interval, if want to improve, we can add some more month data for feature Extraction</p>

###### Lets see some of predicated outputs

In [59]:
y_test[:10]

[34.0, 53.0, 50.0, 62.0, 77.0, 70.0, 100.0, 96.0, 111.0, 117.0]

In [58]:
lgbm_test_predictions[:10]

[26.0, 32.0, 35.0, 49.0, 53.0, 61.0, 73.0, 72.0, 91.0, 96.0]

######  Observation:-
quite less than actuals pickups count, its still ok.
<p>We will try to solve using Deep learning model like LSTM in future</p>

# Thanks for reading my work